In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import os
import re
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
import nltk
nltk.download()
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


LookupError: ignored

Reusing dataset amazon_polarity (/home/student/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/35b3f5f426863b55dda2d5dc0da2d3714c7b0e5ba64bc4f1415ac9291ffdb234)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
RAW_interactions = pd.read_csv("/content/drive/MyDrive/archive/RAW_interactions.csv")

In [ ]:
RAW_interactions["rating"].value_counts()

5    816364
4    187360
0     60847
3     40855
2     14123
1     12818
Name: rating, dtype: int64

In [ ]:
RAW_interactions.isna().value_counts()

user_id  recipe_id  date   rating  review
False    False      False  False   False     1132198
                                   True          169
dtype: int64

In [ ]:
RAW_interactions = RAW_interactions.dropna()

In [ ]:
len(RAW_interactions)

1132198

In [ ]:
RAW_interactions[RAW_interactions["rating"] !=0].value_counts()

user_id     recipe_id  date        rating  review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
2002372706  63786      2018-12-20  5       Finally I found a no nonsense fajita recipe                                                                                                                                                                                     

In [ ]:
RAW_interactions = RAW_interactions[RAW_interactions["rating"] !=0]

In [ ]:
RAW_interactions["rating"].value_counts()

5    816229
4    187333
3     40852
2     14122
1     12815
Name: rating, dtype: int64

In [ ]:
def preprocess_text(dataset):

    def to_lower(text):
        low_text= text.lower()
        return low_text

    def remove_numbers(text):
        remove= re.sub(r'\d+', '', text)
        return remove

    def remove_punktuation(text):
        punct = re.sub(r"[^\w\s\d]","", text)
        return punct


    
    dataset["review"]=dataset["review"].apply(lambda x:to_lower(x))
    dataset["review"]=dataset["review"].apply(lambda x:remove_numbers(x))
    dataset["review"]=dataset["review"].apply(lambda x:remove_punktuation(x))
 
    return dataset["review"]

In [ ]:
RAW_interactions['review']=preprocess_text(RAW_interactions)

In [ ]:
RAW_interactions

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,great with a salad cooked on top of stove for ...
1,1293707,40893,2011-12-21,5,so simple so delicious great for chilly fall e...
2,8937,44394,2002-12-01,4,this worked very well and is easy i used not ...
3,126440,85009,2010-02-27,5,i made the mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,made the cheddar bacon topping adding a sprink...
...,...,...,...,...,...
1132360,2002357020,82303,2018-12-05,5,delicious quick thick chocolate sauce with ing...
1132363,583662,386618,2009-09-29,5,these were so delicious my husband and i trul...
1132364,157126,78003,2008-06-23,5,wow sometimes i dont take the time to rate re...
1132365,53932,78003,2009-01-11,4,very good i used regular port as well the gr...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(RAW_interactions["review"][0:250000]
                                                    , RAW_interactions["rating"][0:250000],random_state = 42,
                                                   test_size = 0.10)
X_train.shape,X_test.shape,y_train.shape

((225000,), (25000,), (225000,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=STOPWORDS)
vectorizer = vectorizer.fit(RAW_interactions.review)

In [ ]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [ ]:
X_train_vectors = vectorizer.transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

<225000x272182 sparse matrix of type '<class 'numpy.float64'>'
	with 5467010 stored elements in Compressed Sparse Row format>

In [ ]:
nb_tdfif = MultinomialNB().fit(X_train_vectors, y_train)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
nb_tdfif

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
predicts = nb_tdfif.predict(X_train_vectors)


In [ ]:
predicts

array([5, 5, 5, ..., 5, 5, 5])

In [ ]:
print(classification_report(y_train, predicts))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00      2603
           2       0.00      0.00      0.00      2965
           3       0.00      0.00      0.00      8592
           4       0.82      0.00      0.00     39477
           5       0.76      1.00      0.86    171363

    accuracy                           0.76    225000
   macro avg       0.32      0.20      0.17    225000
weighted avg       0.72      0.76      0.66    225000



In [ ]:

predicts =nb_tdfif.predict(X_test_vectors)
print(classification_report(y_test, predicts))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       310
           2       0.00      0.00      0.00       310
           3       0.00      0.00      0.00       931
           4       0.50      0.00      0.00      4380
           5       0.76      1.00      0.87     19069

    accuracy                           0.76     25000
   macro avg       0.25      0.20      0.17     25000
weighted avg       0.67      0.76      0.66     25000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
all_data=[RAW_interactions["review"], RAW_interactions["rating"]]
X_vectors = vectorizer.transform(all_data[0])
predicts_all_data =nb_tdfif.predict(X_vectors)


AttributeError: ignored

In [ ]:
print(classification_report(all_data[1], predicts_all_data))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00     12815
           2       0.00      0.00      0.00     14122
           3       0.00      0.00      0.00     40852
           4       0.37      0.00      0.00    187333
           5       0.76      1.00      0.86    816229

    accuracy                           0.76   1071351
   macro avg       0.23      0.20      0.17   1071351
weighted avg       0.65      0.76      0.66   1071351



In [ ]:
from sklearn.naive_bayes import MultinomialNB
bayes_pipe = Pipeline([
 ("tfidf", TfidfVectorizer()),
 ("nb", MultinomialNB())
])


In [ ]:
X_train.shape,X_test.shape,y_train.shape

((225000,), (25000,), (225000,))

In [ ]:
y_test

40762     4
68490     5
4137      5
127014    4
182600    4
         ..
84512     5
253474    5
106356    5
146014    4
214355    5
Name: rating, Length: 25000, dtype: int64

In [ ]:
bayes_pipe.fit(X_train, y_train)

bayes_params = {
    "tfidf__max_features":(1000,5000),
    "tfidf__ngram_range":((2,2), (3,3)),
}

gs_bayes = GridSearchCV(bayes_pipe, param_grid=bayes_params, cv = 3, verbose =1, n_jobs = 4)

In [ ]:
gs_bayes

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [ ]:

train_df=pd.json_normalize(train_data)

test_df=pd.json_normalize(test_data)

In [ ]:

x_train, y_train = preprocess_text(train_df)
x_test, y_test = preprocess_text(test_df)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
bayes_pipe = Pipeline([
 ("tfidf", TfidfVectorizer()),
 ("nb", MultinomialNB())
])



In [ ]:
bayes_pipe.fit(x_train, y_train)

bayes_params = {
    "tfidf__max_features":(1000,5000),
    "tfidf__ngram_range":((2,2), (3,3)),
}

gs_bayes = GridSearchCV(bayes_pipe, param_grid=bayes_params, cv = 3, verbose =1, n_jobs = -1)

In [ ]:
frames = [train_df,test_df]
dataset = pd.concat(frames)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=STOPWORDS)
vectorizer = vectorizer.fit(dataset.content)

In [ ]:
X_train_vectors = vectorizer.transform(train_df.content)
X_test_vectors = vectorizer.transform(test_df.content)

In [ ]:
nb_tdfif = MultinomialNB().fit(X_train_vectors, train_df.label)

In [ ]:
nb_tdfif

MultinomialNB()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predicts = nb_tdfif.predict(X_train_vectors)
print(classification_report(train_df.label, predicts))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85   1800000
           1       0.86      0.83      0.84   1800000

    accuracy                           0.85   3600000
   macro avg       0.85      0.85      0.85   3600000
weighted avg       0.85      0.85      0.85   3600000



In [ ]:
X_test_vectors = vectorizer.transform(test_df.content)
predicts =nb_tdfif.predict(X_test_vectors)
print(classification_report(test_df.label, predicts))

              precision    recall  f1-score   support

           0       0.82      0.84      0.83    200000
           1       0.84      0.81      0.82    200000

    accuracy                           0.82    400000
   macro avg       0.83      0.82      0.82    400000
weighted avg       0.83      0.82      0.82    400000



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re